In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pydicom

In [3]:
import sys
sys.path.insert(0,'/content/drive/')
import os

# TODO: Fill in the Google Drive path where you uploaded the files 
#GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = './EECS545'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab/final_project/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['COVID_combined', 'Normal', 'maskHE2_labels.csv', 'test', 'test_labels.csv', 'validation', 'train_labels.csv', 'val_labels.csv', 'ReadMe.gdoc', 'training.ipynb', 'training', 'train_dataset.npy', 'valid_dataset.npy', 'train_dataset_jiahong.npy', 'valid_dataset_jiahong.npy', 'Model_LungSegmentation', 'Model_Without_Lungseg', 'Copy of Siamese_Learning.ipynb', 'images_nomask', 'HE_output2.zip', 'Things to be done for Final Project.gdoc', 'Copy of Training_VGG16_CrossValidation_WithLungMasking_1.ipynb', 'Training_VGG16_CrossValidation_WithLungMasking.ipynb', 'Siamese_Learning.ipynb', 'grad_labels.csv', 'create_label.ipynb', 'Matching_code']


In [4]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image

import copy

In [5]:
training_transforms = transforms.Compose([
    transforms.Resize(336), # maybe better to factor out resize step
    transforms.CenterCrop(320),
    transforms.ToTensor()
])
validation_transforms = transforms.Compose([
    transforms.Resize(336), # maybe better to factor out resize step
    transforms.CenterCrop(320),
    transforms.ToTensor()
])

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [7]:
train_csv = os.path.join(GOOGLE_DRIVE_PATH,'train_labels.csv')
val_csv = os.path.join(GOOGLE_DRIVE_PATH,'val_labels.csv')
test_csv = os.path.join(GOOGLE_DRIVE_PATH,'test_labels.csv')

class_numbers = 2

test_path = os.path.join(GOOGLE_DRIVE_PATH,"test")
train_path = os.path.join(GOOGLE_DRIVE_PATH,"training")
val_path = os.path.join(GOOGLE_DRIVE_PATH,"validation")

batch_size = 8

In [8]:
# PyTorch modules
import torch
from torch import nn 
from torch.utils import data 
import torch.nn.functional as F 
from torchvision import transforms, models

# other modules
import os
import pandas as pd
import numpy as np
from PIL import Image
import random
import pydicom
import cv2
class CheXpert_Dataset(data.Dataset):
  
    def __init__(self, patient_table, image_dir, transform=None):
 
        self.patient_table = patient_table
        self.image_dir = image_dir # note, images are already 8-bit, full-sized JPEGs
        self.transform = transform
        if self.transform is None:
            self.transform = transforms.Compose([transforms.ToTensor()])
 
    def __len__(self):
        return len(self.patient_table)
 
    def __getitem__(self, idx): 
        
        patient_table = self.patient_table
        # separate tables for abnormal lung labels 0 and 1
        table_A = patient_table[patient_table['encoded_labels'] == 0]
        table_B = patient_table[patient_table['encoded_labels'] == 1]

        # goal is 50:50 distribution of change vs no change
        change_binary = random.randint(0,1) 

        # if no change 
        if change_binary == 0:

            # select two random images 
            table_A_or_B = random.randint(0,1)

            if table_A_or_B == 0:
                pick0 = random.choice(range(len(table_A)))
                pick1 = random.choice(range(len(table_A)))
                img0 = Image.open(self.image_dir + '/' + table_A.iloc[pick0]['Images'])
                img1 = Image.open(self.image_dir + '/' + table_A.iloc[pick1]['Images'])

                abnormal_lung_0 = 0
                abnormal_lung_1 = 0

            if table_A_or_B == 1:
                pick0 = random.choice(range(len(table_B)))
                pick1 = random.choice(range(len(table_B)))
                img0 = Image.open(self.image_dir + '/' + table_B.iloc[pick0]['Images'])
                img1 = Image.open(self.image_dir + '/' + table_B.iloc[pick1]['Images'])

                abnormal_lung_0 = 1
                abnormal_lung_1 = 1

        # if change -- note: direction of change does not matter in this step
        if change_binary == 1: 

            pick0 = random.choice(range(len(table_A)))
            pick1 = random.choice(range(len(table_B)))
            img0 = Image.open(self.image_dir + '/'+ table_A.iloc[pick0]['Images'])
            img1 = Image.open(self.image_dir + '/' + table_B.iloc[pick1]['Images'])

            abnormal_lung_0 = 0
            abnormal_lung_1 = 1

        # 0 for no change, 1 for change (difference between labels)
        if abnormal_lung_0 == abnormal_lung_1:
            change_label = 0
        else:
            change_label = 1
 
        meta = {"path0": patient_table.iloc[pick0]['Images'],
                "path1": patient_table.iloc[pick1]['Images'],
                "abnormal_lung_0": abnormal_lung_0,
                "abnormal_lung_1": abnormal_lung_1,
                }

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1) 

        return img0, img1, change_label, meta

In [9]:
class SiameseNetwork_DenseNet121(nn.Module):

    def __init__(self):
        super(SiameseNetwork_DenseNet121, self).__init__()
        self.cnn0 = torch.hub.load('pytorch/vision:v0.5.0', 'densenet121', pretrained=True)
        # map last layer from 1000 node outcome to three nodes
        #self.cnn0.fc = nn.Linear(1000, 3) # mapping input image to a 3D space

    def forward_once(self, x):
        output = self.cnn0(x)
        return output

    def forward(self, input0, input1):
        output0 = self.forward_once(input0)
        output1 = self.forward_once(input1)
        return output0, output1
  

 

In [10]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    Modified from: https://hackernoon.com/facial-similarity-with-siamese-networks-in-pytorch-9642aa9db2f7
    """ 

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output0, output1, label):
        euclidean_distance = F.pairwise_distance(output0, output1)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive


In [11]:
class MSELoss(torch.nn.Module):
    """
    MSE loss
    """ 

    def __init__(self):
        super(MSELoss, self).__init__()

    def forward(self, output0, output1, label):
        euclidean_distance = F.pairwise_distance(output0, output1)
    
        return F.mse_loss(euclidean_distance,label)

In [12]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torch.autograd import Variable
  
# other modules 
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import statistics
import pickle
from tqdm import tqdm
working_path = '/content/drive/My Drive/Colab/final_project/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
training_table = pd.read_csv(train_csv)
validation_table = pd.read_csv(val_csv)
table_A = validation_table[validation_table['encoded_labels'] == 1]

In [14]:
img0 = Image.open(val_path + '/'+ table_A.iloc[0]['Images'])

In [14]:
training_siamese_dataset = CheXpert_Dataset(patient_table = training_table, 
                                        image_dir = train_path, 
                                        transform = training_transforms)
 
training_dataloader = torch.utils.data.DataLoader(training_siamese_dataset, 
                                                  batch_size=batch_size, 
                                                  shuffle=True)

In [15]:
validation_siamese_dataset = CheXpert_Dataset(patient_table = validation_table, 
                                        image_dir = val_path, 
                                        transform = validation_transforms)
 
validation_dataloader = torch.utils.data.DataLoader(validation_siamese_dataset, 
                                                  batch_size=batch_size, 
                                                  shuffle=True)

In [ ]:
def siamese_training(training_dataloader, validation_dataloader, working_path, output_folder_name, learning_rate = 0.00002):
    '''
    - Implements siamese network training/validation with return of model
    - Implementation uses early stopping, saving the model with the best validation loss
 
    Arguments
    - training and validation dataloader objects
    - output_folder_name: directory to save model and annotations
    - learning_rate: for Adam optimizer
    ''' 
    net = SiameseNetwork_DenseNet121().cuda()
    criterion = ContrastiveLoss()
    criterion.margin = 50.0  # contrastive loss function margin
    optimizer = optim.Adam(net.parameters(),lr = learning_rate)
 
    # Initialization
    num_epochs = 50
    training_losses, validation_losses = [], []

    # Early stopping initialization
    epochs_no_improve = 0
    max_epochs_stop = 10  # "patience" - number of epochs with no improvement in validation loss after which training stops
    validation_loss_min = np.Inf
    history = []

    output_dir = GOOGLE_DRIVE_PATH + 'Matching_code'
    #os.mkdir(output_dir)
    f = open(output_dir + "/history_training_50epoch_p10.txt", 'a+')
    f.write(datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n" + "Training starting now...\n")
    f.close()

    for epoch in range(0, num_epochs):
        print("epoch training started...")

        # keep track of training and validation loss each epoch
        training_loss = 0
        validation_loss = 0

        # keep track of euclidean_distance and label history each epoch
        training_euclidean_distance_history = []
        training_label_history = []
        validation_euclidean_distance_history = []
        validation_label_history = []

        # model set to train
        net.train()
          
        # training loop
        for i, data in tqdm(enumerate(training_dataloader, 0)):
            # train neural network
            img0, img1, label, meta = data
            img0 = np.repeat(img0, 3, 1) # repeat grayscale image in 3 channels (DenseNet requires 3-channel input) 
            img1 = np.repeat(img1, 3, 1)
            img0, img1, label = Variable(img0).cuda(), Variable(img1).cuda(), Variable(label).cuda()  # send tensors to the GPU
            optimizer.zero_grad() # clear gradients
            output0, output1 = net(img0, img1)
            loss_contrastive = criterion(output0, output1, label.float())
            loss_contrastive.backward()
            optimizer.step()

            # keep track of training loss
            training_loss += loss_contrastive.item()

            # save euclidean distance and label history 
            net.eval()
            output0, output1 = net.forward(img0, img1)
            net.train()
            euclidean_distance = F.pairwise_distance(output0, output1)
            euclid_tmp = torch.Tensor.numpy(euclidean_distance.detach().cpu())
            training_euclidean_distance_history.extend(euclid_tmp)

            label_tmp = torch.Tensor.numpy(label.cpu())
            training_label_history.extend(label_tmp)

            print("training loop " + str(i) + " completed")

        else:
            print("validation started...")
            #turn off gradients for validation
            with torch.no_grad(): 
                net.eval() # set evaluation mode

                # determine validation loss
                for j, data2 in tqdm(enumerate(validation_dataloader, 0)):
                    img0, img1, label, meta = data2
                    img0 = np.repeat(img0, 3, 1) # repeat grayscale image in 3 channels (DenseNet requires 3-channel input) 
                    img1 = np.repeat(img1, 3, 1)
                    img0, img1, label = Variable(img0).cuda(), Variable(img1).cuda(), Variable(label).cuda()
                    output0, output1 = net.forward(img0, img1)
                    loss_contrastive = criterion(output0, output1, label.float())
                    validation_loss += loss_contrastive.item()
                     
                    # save euclidean distance and label history 
                    euclidean_distance = F.pairwise_distance(output0, output1)                
                    euclid_tmp = torch.Tensor.numpy(euclidean_distance.detach().cpu())
                    validation_euclidean_distance_history.extend(euclid_tmp)

                    label_tmp = torch.Tensor.numpy(label.cpu())
                    validation_label_history.extend(label_tmp)

                    print("validation loop " + str(j) + " completed")

            # calculate average training and validation losses (averaged across batches for the epoch)
            training_loss_avg = training_loss/len(training_dataloader)
            validation_loss_avg = validation_loss/len(validation_dataloader)

            # Save the model if validation loss decreases
            if validation_loss_avg < validation_loss_min:
                # save model
                torch.save(net.state_dict(), output_dir + "/SNN_densenet121_model_50epoch_p10.pth")
                # track improvement
                epochs_no_improve = 0
                validation_loss_min = validation_loss_avg
                best_epoch = epoch
 
            # Otherwise increment count of epochs with no improvement
            else: 
                epochs_no_improve += 1 
                # Trigger EARLY STOPPING
                if epochs_no_improve >= max_epochs_stop:
                    print(f'\nEarly Stopping! Total epochs (starting from 0): {epoch}. Best epoch: {best_epoch} with loss: {validation_loss_min:.2f}')
                    # Load the best state dict (at the early stopping point)
                    net.load_state_dict(torch.load(output_dir + "/SNN_densenet121_model_50epoch_p10.pth"))
                    # attach the optimizer
                    net.optimizer = optimizer

                    # save history with pickle
                    with open(output_dir + "/history_training_50epoch_p10.pckl", "wb") as f:
                        pickle.dump(history, f)

                    f = open(output_dir + "/history_training_50epoch_p10.txt", 'a+')
                    f.write(datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n" +
                            "Early stopping! Total epochs (starting from 0): {:.0f}\n".format(epoch) +
                            "Best epoch: {:.0f}\n".format(best_epoch) +
                            "Validation loss at best epoch: {:.3f}\n".format(validation_loss_min)
                            )
                    f.close()

                    return net, history, output_dir # break the function

        # after each Epoch

        # append to lists for graphing
        training_losses.append(training_loss_avg)
        validation_losses.append(validation_loss_avg)

        # training euclidean distance stats

        # extract euclidean distances if label is 0 or 1
        euclid_if_0 = [b for a, b in zip(training_label_history, training_euclidean_distance_history) if a == 0]
        euclid_if_1 = [b for a, b in zip(training_label_history, training_euclidean_distance_history) if a == 1]
        euclid_if_0 = np.array(euclid_if_0).tolist()
        euclid_if_1 = np.array(euclid_if_1).tolist()
        
        # summary statistics for euclidean distances
        mean_euclid_0t = statistics.mean(euclid_if_0) 
        std_euclid_0t = statistics.pstdev(euclid_if_0) # population stdev
        mean_euclid_1t = statistics.mean(euclid_if_1)
        std_euclid_1t = statistics.pstdev(euclid_if_1) # population stdev
        euclid_diff_t = mean_euclid_1t - mean_euclid_0t

        # validation euclidean distance stats

        # extract euclidean distances if label is 0 or 1
        euclid_if_0 = [b for a, b in zip(validation_label_history, validation_euclidean_distance_history) if a == 0]
        euclid_if_1 = [b for a, b in zip(validation_label_history, validation_euclidean_distance_history) if a == 1]
        euclid_if_0 = np.array(euclid_if_0).tolist()
        euclid_if_1 = np.array(euclid_if_1).tolist()
        
        # summary statistics for euclidean distances
        mean_euclid_0v = statistics.mean(euclid_if_0) 
        std_euclid_0v = statistics.pstdev(euclid_if_0) # population stdev
        mean_euclid_1v = statistics.mean(euclid_if_1)
        std_euclid_1v = statistics.pstdev(euclid_if_1) # population stdev
        euclid_diff_v = mean_euclid_1v - mean_euclid_0v

        # store in history list
        history = [training_losses, validation_losses, euclid_diff_t, euclid_diff_v]
 
        # save history with pickle
        with open(output_dir + "/history_training_50epoch_p10.pckl", "wb") as f:
            pickle.dump(history, f)
 
        print("Epoch number: {:.0f}\n".format(epoch),
            "Training loss: {:.3f}\n".format(training_loss_avg),
            "Validation loss: {:.3f}\n".format(validation_loss_avg),
            "\nTraining \nLabel0 euclidean distance mean: {:.3f}\n".format(mean_euclid_0t),
            "Label0 euclidean distance std: {:.3f}\n".format(std_euclid_0t),
            "Label1 euclidean distance mean: {:.3f}\n".format(mean_euclid_1t),
            "Label1 euclidean distance std: {:.3f}\n".format(std_euclid_1t),
            "Euclidean distance mean diff: {:.3f}\n".format(euclid_diff_t),
            "\nValidation \nLabel0 euclidean distance mean: {:.3f}\n".format(mean_euclid_0v),
            "Label0 euclidean distance std: {:.3f}\n".format(std_euclid_0v),
            "Label1 euclidean distance mean: {:.3f}\n".format(mean_euclid_1v),
            "Label1 euclidean distance std: {:.3f}\n".format(std_euclid_1v),
            "Euclidean distance mean diff: {:.3f}\n".format(euclid_diff_v)
            )

        # write history to file
        f = open(output_dir + "/history_training_50epoch_p10.txt", 'a+')
        f.write(datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n" +
            "Epoch number: {:.0f}\n".format(epoch) +
            "Training loss: {:.3f}\n".format(training_loss_avg) +
            "Validation loss: {:.3f}\n".format(validation_loss_avg) +
            "\nTraining \nLabel0 euclidean distance mean: {:.3f}\n".format(mean_euclid_0t) +
            "Label0 euclidean distance std: {:.3f}\n".format(std_euclid_0t) +
            "Label1 euclidean distance mean: {:.3f}\n".format(mean_euclid_1t) +
            "Label1 euclidean distance std: {:.3f}\n".format(std_euclid_1t) +
            "Euclidean distance mean diff: {:.3f}\n".format(euclid_diff_t) +
            "\nValidation \nLabel0 euclidean distance mean: {:.3f}\n".format(mean_euclid_0v) + 
            "Label0 euclidean distance std: {:.3f}\n".format(std_euclid_0v) + 
            "Label1 euclidean distance mean: {:.3f}\n".format(mean_euclid_1v) + 
            "Label1 euclidean distance std: {:.3f}\n".format(std_euclid_1v) +
            "Euclidean distance mean diff: {:.3f}\n".format(euclid_diff_v) + "\n"
            )
        f.close()
 
    # Load the best state dict (at the early stopping point)
    net.load_state_dict(torch.load(output_dir + "/SNN_densenet121_model_50epoch_p10.pth"))
    # After training through all epochs attach the optimizer
    net.optimizer = optimizer

    # Return the best model and history
    print(f'\nAll Epochs completed! Total epochs (starting from 0): {epoch}. Best epoch: {best_epoch} with validation loss: {validation_loss_min:.2f}')

    return net, history, output_dir

# siamese training 
net, history, output_dir = siamese_training(training_dataloader = training_dataloader, 
                                            validation_dataloader = validation_dataloader, 
                                            working_path = working_path,
                                            output_folder_name = 'CheXpert_model')
 
# Training/validation learning curves
plt.title("Number of Training Epochs vs. Contrastive Loss")
plt.xlabel("Training Epochs")
plt.ylabel("Contrastive Loss")
plt.plot(range(0, len(history[0])), history[0], label = "Training loss")
plt.plot(range(0, len(history[1])), history[1], label = "Validation loss")
plt.legend(frameon=False)
plt.savefig(output_dir + "/Learning_curve_50epoch_p10.png")
plt.close()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.5.0
0it [00:00, ?it/s]

epoch training started...


1it [00:00,  1.68it/s]

training loop 0 completed


2it [00:01,  1.75it/s]

...

338it [03:11,  1.79it/s]

training loop 337 completed


339it [03:12,  1.78it/s]

training loop 338 completed


340it [03:12,  1.78it/s]

In [ ]:
# Load the test dataset
# test_dataset = SiameseDataset(training_csv=testing_csv,training_dir=testing_dir,
#                                         transform=transforms.Compose([transforms.Resize((105,105)),
#                                                                       transforms.ToTensor()
#                                                                       ]))


test_transforms = transforms.Compose([
    transforms.Resize(336), # maybe better to factor out resize step
    transforms.CenterCrop(320),
    transforms.ToTensor()
])

test_dataset = CheXpert_Dataset(patient_table = test_images, 
                                        image_dir = test_path, 
                                        transform = test_transforms)
 
test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                                  batch_size=batch_size, 
                                                  shuffle=False)

# test the network (should change)
count=0
for i, data in enumerate(test_dataloader,0): 
  x0, x1 , label = data
  concat = torch.cat((x0,x1),0)
  output1,output2 = model(x0.to(device),x1.to(device))

  eucledian_distance = F.pairwise_distance(output1, output2)
    
  if label==torch.FloatTensor([[0]]):
    label="Original Pair Of Signature"
  else:
    label="Forged Pair Of Signature"
    
  imshow(torchvision.utils.make_grid(concat))
  print("Predicted Eucledian Distance:-",eucledian_distance.item())
  print("Actual Label:-",label)
  count=count+1
  if count ==10:
     break